## Calculating likelihoods

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

glasflow is using its own internal version of nflows


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
#reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [3]:
params = ['mchirp','q', 'chieff', 'z']

In [4]:
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            10000, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params)
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')

In [5]:
llks = flow(obsdata,[3,1])
llks

array([1.77619572e-02, 1.10000000e-03, 3.47321586e-05, 6.29553891e-04,
       8.77993859e-01, 2.20000000e-03, 4.07164408e-12, 9.83600000e-01,
       9.22600510e-01, 4.32159394e+00, 8.92000063e-02, 7.34000000e-02,
       6.70076397e-03, 1.31508692e-04, 1.60881138e-03, 6.85500732e-15,
       3.64176205e-04, 1.44529718e-02, 6.27492495e-05, 3.03845148e-13,
       2.25793493e-12, 1.03697854e-09, 1.05729253e-04, 2.68787110e-07,
       5.32942603e-02, 1.00001924e-03, 2.35092711e-01, 4.88123432e-01,
       2.85604168e-01, 1.79800188e-01, 2.64413310e-04, 2.33000000e-02,
       6.03713989e-04, 1.05776259e-11, 2.47662987e-11, 9.53250678e-04,
       8.77552614e-04, 8.15200000e-01, 5.12558384e-08, 1.00000000e+00,
       1.48404095e+00, 1.43000099e-02, 1.66608778e-04, 3.84586601e-06,
       2.36219726e-11, 5.10086623e-12])

In [6]:
np.sum(llks)

11.896166713812777